In [2]:
## idg forked from @levanpon1009 on kaggle.com
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from itertools import chain
from glob import glob
import pandas as pd
import numpy as np
import os

2021-10-07 07:25:24.793059: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


### const

In [3]:
CWD = os.getcwd()
PROJECT_NAME = 'Pneumonia-Classification'
BATCH_SIZE = 16 # from CheXNet paper 
DIM = (256, 256)

In [4]:
print(CWD)

/home/jinen/git/Pneumonia-Classification/task-2-data-processing/src/notebook


### load dataset

In [48]:
DATA_DIR = CWD[:40] + '/data/'
df = pd.read_csv(DATA_DIR+ 'Data_Entry_2017.csv') # TEST '/home/jinen/git/Pneumonia-Classification/data/
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [49]:
data_image_paths = {os.path.basename(x): x for x in glob(os.path.join(DATA_DIR, 'images*', '*', '*.png'))}

In [50]:
df['path'] = df['Image Index'].map(data_image_paths.get)

In [51]:
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,path
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,/home/jinen/git/Pneumonia-Classification/data/...
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,/home/jinen/git/Pneumonia-Classification/data/...
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,/home/jinen/git/Pneumonia-Classification/data/...
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,/home/jinen/git/Pneumonia-Classification/data/...
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,/home/jinen/git/Pneumonia-Classification/data/...


In [52]:
df['Finding Labels'] = df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
labels = [x for x in labels if len(x) > 0]
for label in labels:
    if len(label) > 1:
        df[label] = df['Finding Labels'].map(lambda finding: 1.0 if label in finding else 0.0)
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00000002_000.png,,0,2,81,M,PA,2500,2048,0.171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
train_df, valid_df = train_test_split(df, test_size=0.20, random_state=2018, stratify=df['Finding Labels'].map(lambda x: x[:4]))

In [54]:
train_df['labels'] = train_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)
valid_df['labels'] = valid_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)

In [55]:
df['path'] = df['path'].astype('str')

In [56]:
core_generator = ImageDataGenerator(rescale=1 / 255,
                                    samplewise_center=True,
                                    samplewise_std_normalization=True,
                                    horizontal_flip=True,
                                    vertical_flip=False,
                                    height_shift_range=0.05,
                                    width_shift_range=0.1,
                                    rotation_range=5,
                                    shear_range=0.1,
                                    fill_mode='reflect',
                                    zoom_range=0.15)

training_generator = core_generator.flow_from_dataframe(dataframe=train_df,
                                                  directory=None,
                                                  x_col='path',
                                                  y_col=labels,
                                                  class_mode='raw',
                                                  batch_size=BATCH_SIZE,
                                                  classes=labels,
                                                  target_size=DIM)

validation_generator = core_generator.flow_from_dataframe(dataframe=valid_df,
                                                    directory=None,
                                                    x_col='path',
                                                    y_col=labels,
                                                    class_mode='raw',
                                                    batch_size=BATCH_SIZE,
                                                    classes=labels,
                                                    target_size=DIM)

test_generator = core_generator.flow_from_dataframe(dataframe=valid_df,
                                                    directory=None,
                                                    x_col='path',
                                                    y_col=labels,
                                                    class_mode='raw',
                                                    batch_size=BATCH_SIZE,
                                                    classes=labels,
                                                    target_size=DIM)

/home/jinen/git/Pneumonia-Classification/venv/lib/python3.9/site-packages/keras_preprocessing/image/dataframe_iterator.py:219: UserWarning: `classes` will be ignored given the class_mode="raw"
  warnings.warn('`classes` will be ignored given the class_mode="{}"'


Found 89696 validated image filenames.
Found 22424 validated image filenames.
Found 22424 validated image filenames.
